# *載入模型和Tokenizer*

In [ ]:
from huggingface_hub import notebook_login
notebook_login("")

In [ ]:
!pip install datasets

In [ ]:
!pip install trl
# !pip install --upgrade bitsandbytes

In [ ]:
!pip install -U bitsandbytes

In [ ]:
import torch
from datasets import Dataset , load_dataset
from transformers import (AutoTokenizer,
              AutoModelForCausalLM,
              DataCollatorForSeq2Seq,
              TrainingArguments,
              Trainer,
              BitsAndBytesConfig,
              pipeline,
              logging,
              TextStreamer)

In [ ]:
from peft import LoraConfig , PeftModel , get_peft_model , prepare_model_for_kbit_training
from trl import SFTTrainer
import os , wandb

In [ ]:
model_name = "meta-llama/Llama-3.2-1B"

In [ ]:
dataset_name = "scooterman/guanaco-llama3-1k"

In [ ]:
bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_use_double_quant=False)


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=bnb_config,
                # devize_map = {"":0}

)

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True #自動添加結束標記

In [ ]:
dataset = load_dataset(dataset_name , split = "train")
dataset["text"][0]

# *wandb*

In [ ]:
!pip install wandb
wandb.login(key = "")

In [ ]:
run = wandb.init(
    project = "test",
    job_type = "train"
)

In [ ]:
# 計算參數數量
def print_trainable_parameters(model):
  trainable_params = 0
  all_params = 0
  for _ , param in model.named_parameters():
    all_params += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"訓練參數量:{trainable_params}||總共參數量:{all_params}||訓練參數占比:{100*(trainable_params / all_params):.2f}"
  )

# *LoRA與訓練參數配置*

In [ ]:
peft_config = LoraConfig(
    r = 8,
    lora_alpha = 16, # 可設定為r的兩倍
    # scaling = alpha / r ,LoRA權重值越大影響越大
    # weight += (lora_B @ lora_A) * scaling
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM",
    target_modules = ["q_proj","v_proj"]
)

In [ ]:
training_arguments = TrainingArguments(
    output_dir = "./output",
    num_train_epochs = 5,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 2, # 梯度保存,2步更新一次
    optim = "paged_adamw_8bit",
    save_steps = 100,
    logging_steps = 30,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    fp16 = False,
    bf16 = False,
    max_grad_norm = 0.3, # 最大梯度
    max_steps = 1,
    warmup_ratio = 0.3, # 前30%訓練步數會逐漸提高學習率
    group_by_length = True, # 按序列長度分組,提高訓練效率
    lr_scheduler_type = "linear",
    report_to = "wandb",
)

# *模型微調*

In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    peft_config = peft_config,
    tokenizer = tokenizer,
    dataset_text_field = "text",
    args = training_arguments,
    packing = False
)

# *開始訓練*

In [ ]:
trainer.train()

In [ ]:
model = get_peft_model(model , peft_config)

print_trainable_parameters(model)

訓練參數量:851968||總共參數量:750127104||訓練參數占比:0.1135764852992167
